## Introduction

Detect fraudulent emails.

In [ ]:
# Enter the s3 path of the input data file
s3_source_file="s3://bucket/mycsv.csv"

In [ ]:
# Enter the s3 path of the output data file
bucket = 'aegovansagemaker' # Replace with your own bucket name if needed, this is the destination bucket
prefix = 'emailfraud/sagemaker' 

## Setup

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region. 
- The IAM role ARN used to give SageMaker access to your data. It can be fetched using the **get_execution_role** method from sagemaker python SDK.

In [ ]:
!pip install scikit-plot==0.3.7
!pip install numpy==1.15.1
!pip install matplotlib==2.7.3
!pip install scikit-learn==0.20.0
!pip install nltk==3.3
!pip install pandas==0.22.0
!pip install sagemaker==1.10.1  


In [2]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = get_execution_role()


### Download data from S3

In [ ]:
local_tmp_dir="tmp"
inputfile="{}/data.csv".format(local_tmp_dir)
!mkdir -p local_tmp_dir

In [ ]:

!aws s3 cp $s3_source_file $inputfile

In [ ]:
!head $inputfile

### Prepare the dataset

Reformat the file as expected by blazing text with no header

In [ ]:
import csv
import nltk


def get_data(inputfile):
    nltk.download('punkt')
    data = []
    with open(inputfile, "r") as i:
        csv_reader = csv.reader(i, delimiter=",", quotechar='"')
        # Ignore head
        next(csv_reader)
        for l in csv_reader:
            raw_label= l[5]
            label="__label__{}".format(raw_label)
            # Features - Comments, Make, Model , Year, Price
            text = "{} {} {} {} {}".format(l[4], l[11], l[12], l[13], l[14]).replace("\n", "")
            tokens = nltk.word_tokenize(text.lower())      
            data.append({"label" : raw_label, "text":text, "tokens": tokens})

    return data
        


   
        
    

In [ ]:

data_list = get_data(inputfile)


#### Analyse the dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
data = pd.DataFrame(data_list)


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 10000)
pd.set_option('display.max_rows', 100)

In [ ]:
print("Total no of records in the dataset {}".format(data.shape[0]))

In [ ]:
data.head(n=3)

Let us inspect the dataset and the classes to get some understanding about how the data and the label is provided in the dataset. 

In [ ]:
data["label"].value_counts().plot.pie(autopct='%.2f',figsize=(5, 5), colors=["green","yellow"])
plt.title = "Fraud class distribution"
plt.show()

In [ ]:
num_classes = data["label"].nunique()

Split Data into Train, Test and Validation

In [ ]:
from sklearn.model_selection import train_test_split, learning_curve
train_test, validation = train_test_split(data, test_size = 0.2, random_state = 777)
train, test = train_test_split(train_test, test_size = 0.2, random_state = 777)

View the class distribution for the train and validation set

In [ ]:
fig, ax = plt.subplots( 1,3, figsize=(15,5))
axis=ax[0]
train["label"].value_counts().plot.pie(autopct='%.2f', colors=["green","yellow"], ax=axis)
axis.set_title("Train set fraud class distribution")

axis=ax[1]
validation["label"].value_counts().plot.pie(autopct='%.2f', colors=["green","yellow"], ax=axis)
axis.set_title("Validation set fraud class distribution")


axis=ax[2]
test["label"].value_counts().plot.pie(autopct='%.2f', colors=["green","yellow"], ax=axis)
axis.set_title("Test set fraud class distribution")
plt.show()

#### Prepare the dataset file for Blazing text
The first column should be the label formatted as \__label__<labelname\>

In [ ]:
import csv

def write_formatted_file(data, outputfile):
    with open(outputfile , "w") as o:
        for index, row in data.iterrows():
            label="__label__{}".format(row["label"])
            token_sep_space=" ".join(row["tokens"])
            o.write("{} {}\n".format(label, token_sep_space))

In [ ]:
from sklearn.model_selection import train_test_split, learning_curve


formatted_train_file="{}/train.txt".format(local_tmp_dir)
formatted_validation_file="{}/validation.txt".format(local_tmp_dir)
formatted_test_file="{}/test.txt".format(local_tmp_dir)



write_formatted_file(train, formatted_train_file)
write_formatted_file(validation, formatted_validation_file)
write_formatted_file(test, formatted_test_file)

This is how the formatted file looks like.

In [ ]:
!head $formatted_train_file

The data preprocessing cell might take a minute to run. After the data preprocessing is complete, we need to upload it to S3 so that it can be consumed by SageMaker to execute training jobs. We'll use Python SDK to upload these two files to the bucket and prefix location that we have set above.   

In [ ]:
%%time

train_channel = prefix + '/train'
validation_channel = prefix + '/validation'

sess.upload_data(path=formatted_train_file, bucket=bucket, key_prefix=train_channel)
sess.upload_data(path=formatted_validation_file, bucket=bucket, key_prefix=validation_channel)

s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)


Next we need to setup an output location at S3, where the model artifact will be dumped. These artifacts are also the output of the algorithm's traning job.

In [ ]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

## Training
Now that we are done with all the setup that is needed, we are ready to train our object detector. To begin, let us create a ``sageMaker.estimator.Estimator`` object. This estimator will launch the training job.

In [9]:
region_name = boto3.Session().region_name
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'blazingtext')

## Training the BlazingText model for supervised text classification

In [ ]:
bt_model = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.c4.4xlarge',
                                         train_volume_size = 30,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

Please refer to [algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext_hyperparameters.html) for the complete list of hyperparameters.

In [ ]:
bt_model.set_hyperparameters(mode="supervised",
                            epochs=600,
                            min_count=2,
                            learning_rate=0.01,
                            vector_dim=150,
                            early_stopping=True,
                            patience=50,
                            min_epochs=5,
                            subwords=False,
                            word_ngrams=3)

Now that the hyper-parameters are setup, let us prepare the handshake between our data channels and the algorithm. To do this, we need to create the `sagemaker.session.s3_input` objects from our data channels. These objects are then put in a simple dictionary, which the algorithm consumes.

In [ ]:
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/plain', s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}

In [ ]:
bt_model.fit(inputs=data_channels, logs=True)

In [7]:
def create_model(trainingjobname, container_name, role):
    import boto3
    from time import gmtime, strftime

    sagemaker_session = sagemaker.Session()
    model_name="blazingtextemailfraud" + '-model-' +  strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    print(model_name)
    desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=trainingjobname)
    model_data = desc['ModelArtifacts']['S3ModelArtifacts']

    print(model_data)

    primary_container = {
        'Image': container_name,
        'ModelDataUrl': model_data
    }

    client = boto3.client('sagemaker')
    create_model_response = client.create_model(
        ModelName = model_name,
        ExecutionRoleArn = role,
        PrimaryContainer = primary_container)
    print(create_model_response['ModelArn'])
    print( model_name)
    return model_name

In [ ]:
%%time

chainer_training_job = bt_model.latest_training_job.name
model_name = create_model(chainer_training_job, container, role)



## Hyperparamter Tuning to optimise even further..

In [ ]:
tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [],
      "ContinuousParameterRanges": [
       
        {
          "MaxValue": ".08",
          "MinValue": ".01",
          "Name": "learning_rate"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "1000",
          "MinValue": "200",
          "Name": "epochs"
        },
          
        {
          "MaxValue": "10",
          "MinValue": "2",
          "Name": "min_count"
        },
           {
          "MaxValue": "400",
          "MinValue": "50",
          "Name": "vector_dim"
        },
          
      ]
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 20,
      "MaxParallelTrainingJobs": 3
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "validation:accuracy",
      "Type": "Maximize"
    }
  }

In [ ]:

     
training_job_definition = {
    "AlgorithmSpecification": {
      "TrainingImage": container,
      "TrainingInputMode": "File"
    },
    "InputDataConfig": [
      {
        "ChannelName": "train",
        "CompressionType": "None",
        "ContentType": "text/plain",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_train_data
          }
        }
      },
      {
        "ChannelName": "validation",
        "CompressionType": "None",
        "ContentType": "text/plain",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_validation_data
          }
        }
      }
    ],
    "OutputDataConfig": {
      "S3OutputPath": "s3://{}/{}/output".format(bucket,prefix)
    },
    "ResourceConfig": {
      "InstanceCount": 2,
      "InstanceType": "ml.c4.4xlarge",
      "VolumeSizeInGB": 10
    },
    "RoleArn": role,
    "StaticHyperParameters": {
        "mode":"supervised",
        "early_stopping":"True",
        "patience":"50",
        "min_epochs":"5"
    },
    "StoppingCondition": {
      "MaxRuntimeInSeconds": 43200
    }
}


In [ ]:
from time import gmtime, strftime

tuning_job_name = "FraudEnquiry-" + strftime("%Y%m%d%H%M%S", gmtime())
smclient = boto3.Session().client('sagemaker')

smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = tuning_job_name,
                                           HyperParameterTuningJobConfig = tuning_job_config,
                                           TrainingJobDefinition = training_job_definition)

#### Get the best training job and create a model

In [5]:

smclient = boto3.Session().client('sagemaker')

tuning_job_details = smclient.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)
tuningjob_status = tuning_job_details["HyperParameterTuningJobStatus"]

if tuningjob_status == "Completed":
    best_training_job_name = tuning_job_details["BestTrainingJob"]["TrainingJobName"]
    print("The best training job is {}".format(best_training_job_name))
else:
    print("The hyperparamter tuning job is in status {}. If the job is inprogress wait until it completes sucessfully ..  ". format(tuningjob_status))

The best training job is blazingtextemail-20181015053131-019-9d381ee2


In [10]:
model_name = create_model(best_training_job_name, container, role)

blazingtextemailfraud-model-2018-10-18-23-03-12
s3://aegovansagemaker/cs_emailfraud/sagemaker/output/blazingtextemail-20181015053131-019-9d381ee2/output/model.tar.gz
arn:aws:sagemaker:us-east-2:324346001917:model/blazingtextemailfraud-model-2018-10-18-23-03-12
blazingtextemailfraud-model-2018-10-18-23-03-12


## Inference using sagemaker batch transform

In [ ]:
batchfileinput="{}/batchtest.json".format(local_tmp_dir)
batchfileoutput="{}/batchtest.output.json".format(local_tmp_dir)


#### Create a json file that is formatted  for blazing text inference
It kind of looks like
```json
{
  "instances": ["hey mate , very interested in this", "does it include a pakc ?", "hi is it working"],
  "configuration": {
    "k": 2
  }
}
```

In [ ]:
%%time

import json
from itertools import islice
import math
import struct
import json

test_data = test

input_records = [" ".join(l["tokens"]) for index, l in test_data.iterrows()]
labels = ["__label__{}".format(l["label"]) for index, l  in test_data.iterrows() ]

input_json = {"instances" : input_records, "configuration": {"k": num_classes}}
with open(batchfileinput , "w") as f:
    f.write(json.dumps(input_json))
                                          

In [ ]:
%%time
import boto3
import sagemaker
import json

fmttime= strftime("%Y-%m-%d-%H-%M-%S", gmtime())
input_key_file="batchvalidation.csv"
input_batch_key="{}/batchTransform/{}_input/{}".format(prefix, fmttime, input_key_file)
input_location = 's3://{}/{}'.format(bucket, input_batch_key)
output_batch_key = "{}/batchTransform/{}_output".format(prefix,fmttime)
output_location = 's3://{}/{}'.format(bucket, output_batch_key)


s3_client = boto3.client('s3')
s3_client.upload_file(batchfileinput, bucket, input_batch_key)

# Initialize the transformer object
transformer =sagemaker.transformer.Transformer(
    base_transform_job_name='Batch-Transform',
    model_name=model_name,
    instance_count=1,
    instance_type='ml.c4.xlarge',
    output_path=output_location
    )
# To start a transform job:
transformer.transform(input_location, content_type='application/json')
# Then wait until transform job is completed
transformer.wait()

# To fetch validation result 
outputkey ='{}/{}.out'.format(output_batch_key, input_key_file)
print(outputkey)
s3_client.download_file(bucket, outputkey, batchfileoutput)



In [ ]:
import json

with open(batchfileoutput) as f:
    predicted = json.load(f)



The output of Blazing text inference looks like this
```json
[{
    'prob': [0.9910072088241577, 0.00901278480887413],
    'label': ['__label__0', '__label__1']
  }, {
    'prob': [0.9973444938659668, 0.002675538882613182],
    'label': ['__label__1', '__label__0']
  }, {
    'prob': [0.9969576001167297, 0.003062397940084338],
    'label': ['__label__0', '__label__1']
  ]
]
```

So lets format it so that we can analyse the performance

In [ ]:
predicted_labels = [ r['label'][r['prob'].index(max(r['prob']))] for r in predicted]
predicted_confidence_score = [ max(r['prob'])/sum(r['prob']) for r in predicted]



Convert the label format \__label__1 to 1 , as the scores work on numeric labels only

In [ ]:
import pandas as pd

y_actual = pd.DataFrame(labels, columns = ["label"]) #== "__label__1"
y_pred = pd.DataFrame(predicted_labels, columns = ["predicted_label"]) #== "__label__1"
y_confidence_score =  pd.DataFrame(predicted_confidence_score, columns = ["predicted_label_confidence_score"])

In [ ]:
import numpy as np
unique_classes = predicted[0]['label']

y_classwise_prob = pd.DataFrame(0, index=np.arange(len(predicted)), columns=unique_classes)
for c in unique_classes:
    y_classwise_prob[c] = pd.DataFrame([ r['prob'][r['label'].index(c)]/sum(r['prob']) for r in predicted])



### Evaluate the performance

In [ ]:
positive_label="__label__1"

In [ ]:


y_display= pd.DataFrame()
y_display["predicted_label"] = y_pred["predicted_label"]
y_display['confidence_score'] = y_confidence_score["predicted_label_confidence_score"]
y_display['score_postive_class'] = y_classwise_prob["__label__1"]
y_display['actual'] = y_actual['label']
y_display.head(n=10)



In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt

skplt.metrics.plot_confusion_matrix(y_actual, y_pred, labels=None, true_labels=None, pred_labels=None, title=None, normalize=False, hide_zeros=False, hide_counts=False, x_tick_rotation=0, ax=None, figsize=None, cmap='Blues', title_fontsize='large', text_fontsize='medium')
plt.show()

In [ ]:
import sklearn
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_recall_fscore_support
import numpy as np

## Precision recall
macro_score = sklearn.metrics.average_precision_score( y_actual, y_classwise_prob[positive_label] ,  pos_label = "__label__1")
f1score = sklearn.metrics.f1_score(y_actual, y_pred , pos_label = positive_label, average='binary')
precision = sklearn.metrics.precision_score(y_actual, y_pred ,  pos_label = positive_label, average='binary')
recall = sklearn.metrics.recall_score( y_actual, y_pred ,  pos_label = positive_label, average='binary')

## Unweighted ROC
acc = accuracy_score(y_actual, y_pred)
fpr, tpr, _ = roc_curve(y_actual,y_classwise_prob[positive_label], pos_label=positive_label)
roc_auc = auc(fpr, tpr)

print("The AUC under precision recall curve is {}".format( macro_score))
print("F1 score {}".format(f1score))
print("Precision score {}".format(precision))
print("Recall score {}".format(recall))
print("Accuracy {}".format( acc))
print("Area Under ROC curve {}".format( roc_auc))



In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt

y_true =  y_actual# ground truth labels
skplt.metrics.plot_roc(y_true, y_classwise_prob, figsize=(10,10))
plt.show()

### Deploy the endpoint
Now that the results look satisfactory, deploy the inference endpoint to serve inference over http

#### Create the endpoint configuration

In [14]:
from time import gmtime, strftime

smclient = boto3.Session().client('sagemaker')

endpoint_config_name = 'FraudDetectionConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = smclient.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])


FraudDetectionConfig-2018-10-18-23-25-01
Endpoint Config Arn: arn:aws:sagemaker:us-east-2:324346001917:endpoint-config/frauddetectionconfig-2018-10-18-23-25-01


#### Create endpoint
This takes 6-7 minutes to complete..

%%time
import time
smclient = boto3.Session().client('sagemaker')


endpoint_name = 'FraudDetection-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = smclient.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = smclient.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = smclient.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

#### Run Sample invocation

In [33]:
import json 

runtime_client = boto3.client('runtime.sagemaker')
def isFraud(json_payload):
    content_type = "application/json"
    payload = json.dumps(json_payload)
    response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType=content_type, 
                                   Body=payload)
    return json.loads(response["Body"].read())

In [35]:
import nltk
sentences = ["this is a .",
            "Can we organise a time to view please."]

# using the same nltk tokenizer that we used during data preparation for training
tokenized_sentences = [' '.join(nltk.word_tokenize(sent)) for sent in sentences]

payload = {"instances" : tokenized_sentences, "configuration": {"k": 2}}

result = isFraud(payload)
print(result)


[{'prob': [0.9939164519309998, 0.0061035798862576485], 'label': ['__label__0', '__label__1']}, {'prob': [1.0000100135803223, 1.0000003385357559e-05], 'label': ['__label__0', '__label__1']}]


#### Demo cleanup
Since this is a demo delete the endpoint

In [37]:
smclient = boto3.Session().client('sagemaker')
smclient.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '7f298191-2130-46b5-ba4c-f9fe8d99252a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7f298191-2130-46b5-ba4c-f9fe8d99252a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 18 Oct 2018 23:37:28 GMT'},
  'RetryAttempts': 0}}